In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
import tempfile
temp_dir = tempfile.TemporaryDirectory()
from casatasks import simobserve, tclean, exportfits
import os
import shutil
from casatools import table
os.environ['MPLCONFIGDIR'] = temp_dir.name

In [ ]:
def load_fits(inFile):
    hdu_list = fits.open(inFile)
    data = hdu_list[0].data
    hdu_list.close()
    return data

def get_band_central_freq(band):
    if band == 3:
        return '100GHz'
    elif band == 6:
        return '250GHz'
    elif band == 9:
        return '650GHz'

def my_asserteq(*args):
    for aa in args[1:]:
        if args[0] != aa:
            raise RuntimeError(f"{args[0]} != {aa}")

def _ms2resolve_transpose(arr):
    my_asserteq(arr.ndim, 3)
    return np.ascontiguousarray(np.transpose(arr, (0, 2,1)))


def ms_to_npz(ms, dirty_cube, datacolumn='CORRECTED_DATA', output_file='test.npz'):
    tb = table()
    tb.open(ms)
    
    #get frequency info from dirty cube
    with fits.open(dirty_cube, memmap=False) as hdulist: 
            npol, nz,nx, ny = np.shape(hdulist[0].data)
            header=hdulist[0].header
    crdelt3 = header['CDELT3']
    crval3 = header['CRVAL3']
    wave = ((crdelt3 * (np.arange(0, nz, 1))) + crval3) #there will be problems, channels       are not of the same width in real data

    vis = tb.getcol(datacolumn)
    vis = np.ascontiguousarray(_ms2resolve_transpose(vis))

    wgt = tb.getcol('WEIGHT')
    wgt = np.repeat(wgt[:,None],128,axis=1)
    #this is to get vis and wgt on the same shape if ms has column weighted_spectrum this       should be different
    wgt = np.ascontiguousarray(_ms2resolve_transpose(wgt))

    uvw = np.transpose(tb.getcol('UVW'))

    np.savez_compressed(output_file,
                    freq = wave,
                    vis= vis, 
                    weight= wgt,
                    polarization=[9,12], 
                    antpos0=uvw,
                    antpos1=tb.getcol('ANTENNA1'),
                    antpos2=tb.getcol('ANTENNA2'),
                    antpos3=tb.getcol('TIME'))

In [ ]:
i = 1
datadir = '/mnt/d/Astro/'
output_dir = 'sims'
output_dir = os.path.join(datadir, output_dir)
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
project = 'sim_{}'.format(i)
name = 'sky_{}.fits'.format(i)
filename = os.path.join(datadir, name)
band = 6
n_px = 300
central_freq = get_band_central_freq(band)
antenna_name = 'alma.cycle9.3.3'
antennalist = os.path.join('/home/michele/GitHub/ALMASim/antenna_config', antenna_name + '.cfg')

In [ ]:
simobserve(
        project=project, 
        skymodel=filename,
        inbright="0.001Jy/pix",
        indirection="J2000 03h59m59.96s -34d59m59.50s",
        incell="0.01arcsec",
        incenter=central_freq,
        inwidth="10MHz",
        setpointings=True,
        integration="10s",
        direction="J2000 03h59m59.96s -34d59m59.50s",
        mapsize=["10arcsec"],
        maptype="hexagonal",
        obsmode="int",
        antennalist=antennalist,
        totaltime="2400s",
        thermalnoise="tsys-atm",
        user_pwv=0.3,
        seed=11111,
        graphics="none",
        verbose=False,
        overwrite=True)
tclean(
        vis=os.path.join(project, "{}.{}.noisy.ms".format(project, antenna_name)),
        imagename=os.path.join(project, '{}.{}'.format(project, antenna_name)),
        imsize=[int(n_px), int(n_px)],
        cell="0.01arcsec",
        phasecenter="",
        specmode="cube",
        niter=0,
        fastnoise=False,
        calcpsf=True,
        pbcor=False
        )
exportfits(imagename=os.path.join(project, '{}.{}.image'.format(project, antenna_name)), 
           fitsimage=os.path.join(output_dir, "dirty_cube_" + str(i) +".fits"), overwrite=True)
exportfits(imagename=os.path.join(project, '{}.{}.skymodel'.format(project, antenna_name)), 
           fitsimage=os.path.join(output_dir, "clean_cube_" + str(i) +".fits"), overwrite=True)

ms_to_npz(os.path.join(project, "{}.{}.noisy.ms".format(project, antenna_name)),
              dirty_cube=os.path.join(output_dir, "dirty_cube_" + str(i) +".fits"),
              datacolumn='CORRECTED_DATA',
              output_file=os.path.join(output_dir, "{}.{}.noisy_".format(project, antenna_name) + str(i) +".npz"))
shutil.rmtree(project)

In [ ]:
clean_data = load_fits(os.path.join(output_dir, "clean_cube_" + str(i) +".fits"))
dirty_data = load_fits(os.path.join(output_dir, "dirty_cube_" + str(i) +".fits"))

print(np.max(clean_data))
print(np.min(clean_data))
print(np.mean(clean_data))
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(np.log10(clean_data[0, 0, :, :]))
ax[1].imshow(dirty_data[0, 0, :, :])
plt.show()

plt.figure(figsize=(10, 5))
plt.hist(np.log10(clean_data.flatten()), bins=100)
plt.show()